# Exercise: Train an image captioning network
---

"*Image Captioning is the process of generating textual description of an image. It uses both Natural Language Processing and Computer Vision to generate the captions*"

In this mandaroty exercise you are implementing an image captioning network. The network will consist of an encoder and a decoder. The encoder is a convolutional neural network, and the decoder is a recurrent neural network. Producing reasonable textual description of an image is a hard task, however with the use of a CNN and a RNN we can start to generate somewhat plausible descriptions. 


Links:
- [Task1: Implement useful functions](#Task1)
- [Task2: Train the recurrent neural network](#Task2)
- [Task3: Generate image captions](#Task3)


Software version:
- Python 3.6
- TensorFlow 1.4.0




---

### Evaluation format ###


You will be guided through the implementation step by step, and you can check your implementation at each step. Note, you will often need to complete all previous steps in order to continue.

In the implementation part, test functions allow you to check your code rapidly. when your code works, you can apply it on part 2 to train and part 3 to generate captions. Part 2, the training process, is slow and can run over night on a typical laptop computer. 

---

### Exercise content


All subtasks that you are to answer are found in this notebook. All implementation should be done in the file "RNN_network.py" found in folder "/sourceFiles/". The skeleton of the program is already implemented and contains things such as:
- Importing data
- How the tensors are connected within the computational graph
- Training framework


As mentioned, an image captioning network consists of an encoder and a decoder. Your task is to implement the decoder (RNN). The images have already been processed through a CNN. The feature vectors are stored in pickle files together with corresponding labels.


During task 1, you will implement all required functionalities for training the image captioning network. In task 2, you will train the network and study how different RNN arcitectures influences the loss. You will generate image captions from images in the validation set in task 3. 



---

### Dataset ###

We will use a dataset called "Common Object in Context" (COCO) 2017. It has ~120,000 training and 5,000 validation images. Every image also includes ~5 captions.



<img src="utils_images/bear.png",width=300>
*Figure 1:*

Captions:

- A big burly grizzly bear is show with grass in the background.
- The large brown bear has a black nose.
- Closeup of a brown bear sitting in a grassy area.
- A large bear that is sitting on grass. 
- A close up picture of a brown bear's face.

---


### Network architecture ###

**Encoder**


Convolutional neural networks have shown to be useful for extracting high level features from images. We will use a pretrained VGG16 network trained on ImageNet. ImageNet consists of 1,2 million images distributed over 1000 classes, and we hope the model have learnt many general features. We are not interested in classifying the 1000 classes, and will change the last fully connected layer with our own. We will use a tanh actiation function to squeeze the values between -1 and 1 similar to the recurrent cells. 

**Decoder**

To be able to convert the high level features from the encoder to natural language, we will construct a recurrent neural network. The output of the encoder will be passed as the initial state to the recurrent cells. The input to the recurret neural network will be word embeddings which we will learn. 

**Loss function**

The words will be considered as separate classes and we shall use cross entropy loss.

**Training vs testing**

When we train the RNN, we will feed in the correct token (word) for every time step, see figure 2a. The words (tokens) are generally unknown, and in test mode, we will need to use our best estimate as input. Example, if the word "brown" has the highest probabillity after the softmax at timestep 2, we will feed in "brown" as input at timestep 3, see figure 2b.


<br>
<br>
<br>





<img src="utils_images/image_captioning_diagram_train.png",width=2200>

<br>
*Figure 2a:* The figure shows an example with 3 recurrent cells stacked in training mode. 

<br>
<br>
<img src="utils_images/image_captioning_diagram_test.png",width=2200>

<br>
*Figure 2b:* The figure shows an example with 3 recurrent cells stacked in test mode. 

---

### Vocabulary ###


To make good decisions in terms of the network architecture, we will need to know the statistics of the captions:
- The number of words in the captions (sequence length) should be considered when chooseing the truncated backpropagation length. 
- To save memory, it is normal to limit the vocabulary size/length. There will be a tradeoff between capturing all words and have a reansonble sized softmax layer. 


Note: The captions have been filtered such that all special characters have been removed. This includes also punctuations and commas. All characters have been changed to lower case.



----
**Vocabulary size**

To make good predictions we need the model to have the abillity to predict frequent words. Figure 3 shows a sorted histogram of the word count for the different words. We can see that the majority of the words are within the 2000 most frequent words. Figure 4 shows the precentage of the words accounted for as a function of the vocabulary size.

<br>

<img src="utils_images/Word_count_hist.png", width=800>
*Figure 3*

<br>

<img src="utils_images/accumulated_Word_count.png",width=800>
*Figure 4*

---


**Truncated backpropagation length**


The sequence length of the recurrent neural network should be able to capture the time(step) dependencies within the captions. Figure 5 shows a histogram of the caption counts as a function of the caption (/sequence) length. Figure 6 shows the precentage of all captions with shorter or equal caption length as a function of caption (/sequence) length. 

<br>
<img src="utils_images/Sequence_lengths_hist.png", width=800>
*Figure 5*

<br>

<img src="utils_images/accumulated_Sequence_length_count.png", width=800>
*Figure 6*


---

**Common words and their tokens**


Every word is assosiated with a token. The words are sorted such that the most frequent words have lower token values. In the table below, you find the six most common words in the dataset. The three first words/tokens have a spescial purpuse:
- "eeee": Indicates the end of a caption.
- "ssss": Start token, first word 
- "UNK": As the vocabulary size is can be smaller than the number of words in the datasset, we change all "unknown" words to the word "UNK"


<br>

<img src="utils_images/vocabulary.png", width=800>





---
### Word embedding  ###


The input to the first rnn layer is the embedded version of the input words. We will define a word embedding matrix with shape [vocabulary_size, embedding_size)]. A single word embedding will be a row vector with length [embedding_size]. The tokens are used to select the correct row within the word embedding matrix. For example, the word "a" will have values in the third row in the embedding matrix (were first row is row zero).


a_emb = wordEmbedding[a_token, :]


The word embedding matrix will be initialized with random values and they will be updated as part of the training process. The goal is that similar words get similar vector representations within the embedding matrix. This will not be part of the assigment, but as a test, the vector representations could be embedded using e.g. t-SNE for plotting in 2D/3D space.



--- 

### Data preparation  ###

Before starting to work on implementing the network aritecture, you should check if you need to download and process data:

- **UIO**: If you work on a UIO IFI computer, we have already processed and made the data avaiable for you on the project disk. The path to this disk is given as default in the code. 

- **Personal computer**: If you plan to work on the assigment on your personal computer, you will need to download and process the data yourself. Follow the steps in notebook "data_preparation.ipynb". Running the data preparation can take several hours due to the size of this real data set. 


The processed data is pickle files holding the following information for every image:
- Path to the jpg image
- Captions
- Captions given as tokens
- The fc7 output from the VGG16 network.




<a id='Task1'></a>

---


# Task 1: Implement useful functions #
See function headings for specification of each function.




In [ ]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2


---

**Task 1a): getInputPlaceholders()** 

To be able to feed data into the computational graph, we can use tf.placeholders. Your task is to implement the function "getInputPlaceholders()". 



In [ ]:
#Import modules
import tensorflow as tf
from sourceFiles import RNN_network 
from unit_tests import unit_tests

#Resetting the tensorflow graph
tf.reset_default_graph()

#Defining dummy variables
VggFc7Size = 4096
truncated_backprop_length = 20

# You should implement this function
xVggFc7, xTokens = RNN_network.getInputPlaceholders(VggFc7Size, truncated_backprop_length)

#Check implementation
unit_tests.checkGetInputPlaceholders(xVggFc7, xTokens)

---

**Task 1b): getInitialState()** 

We will pass the output layer (fc7) from the VGG16 network through a fully connected layer before using it as the initial state of the recurrent cells. Your task is to implement the fully connected layer, using tanh-activation.


In [ ]:
#Import modules
import tensorflow as tf
from sourceFiles import RNN_network 
from unit_tests import unit_tests

#Resetting the tensorflow graph
tf.reset_default_graph()

#Defining dummy variables
VggFc7Size = 4096
xVggFc7 = tf.random_normal(shape=(256, 4096), name='xVggFc7', seed=0)
hidden_state_sizes = 512

# You should implement this function
initial_state = RNN_network.getInitialState(xVggFc7, VggFc7Size, hidden_state_sizes)

#Check implementation
unit_tests.checkGetInitialState(initial_state, hidden_state_sizes)


---

**Task 1c** 

Each word will be represented by a feature vector. The feature vectors are rows in the wordEmbeddingMatrix. Complete getWordEmbeddingMatrix to set up the embedding. 

In [ ]:
#Import modules
import tensorflow as tf
from sourceFiles import RNN_network 
from unit_tests import unit_tests

#Resetting the tensorflow graph
tf.reset_default_graph()

#Defining dummy variables
vocabulary_size = 5000
embedding_size  = 128

# You should implement this function
wordEmbeddingMatrix = RNN_network.getWordEmbeddingMatrix(vocabulary_size, embedding_size)


#Check implementation
unit_tests.checkgetWordEmbeddingMatrix(wordEmbeddingMatrix)

---

**Task 1d** 

Implement the function "getInputs()".
Hint: tf.nn.embedding_lookup and tf.unstack might be useful.  



In [ ]:
#Import modules
import tensorflow as tf
from sourceFiles import RNN_network
from unit_tests import unit_tests

#Resetting the tensorflow graph
tf.reset_default_graph()

#Defining dummy variables
wordEmbeddingMatrix = tf.random_uniform(shape=(1000, 256))
xTokens             = tf.random_uniform(shape=(128, 8), minval=0, maxval=999, dtype=tf.int32)


# You should implement this function
inputs = RNN_network.getInputs(wordEmbeddingMatrix, xTokens)


#Check implementation
unit_tests.checkGetInputs(inputs, wordEmbeddingMatrix, xTokens)

---

**Task 1e** 

After the last RNN cell, we feed the features from the hidden state into a fully connected layer. Your task is to implement the function, getRNNOutputWeights(), which returns the weights to be used in the fully connected layer.

In [ ]:
#Import modules
import tensorflow as tf
from sourceFiles import RNN_network
from unit_tests import unit_tests

#Resetting the tensorflow graph
tf.reset_default_graph()

#Defining dummy variables
hidden_state_sizes = 128
vocabulary_size    = 1000


# You should implement this function
W_hy, b_hy = RNN_network.getRNNOutputWeights(hidden_state_sizes, vocabulary_size)


#Check implementation
unit_tests.checkGetRNNOutputWeights(W_hy, b_hy)


---

**Task 1f** 

Your are to implement a vanailla RNN class. The class shall have a constructor and the function "forward". To speed up inference, we will concatinate the input and the old state matrices and perform a single matrix multiplication. The difference is illustrated in the equation below.

Vanilla RNN:

\begin{align}
h_t &= tanh(x_tW^{hx} + h_{t-1}W^{hh} + b) \\
\\
h_t &= tanh([x_t,h_{t-1}]W + b)
\end{align}



In [ ]:
#Import modules
import tensorflow as tf
from sourceFiles import RNN_network
from unit_tests import unit_tests

#Resetting the tensorflow graph
tf.reset_default_graph()

#Defining dummy variables
hidden_state_sizes = 512
inputSize = 648
ind = 0

# You should implement this function
cell   = RNN_network.RNNcell(hidden_state_sizes, inputSize, ind)

#Check implementation
unit_tests.checkRNNcell(cell)

---

**Task 1g** 

Your are to implement a Gated recurrent units (GRU) class. The class shall have a constructor and the function "forward". To speed up inference, we will concatinate the input and the old state matrices and perform a single matrix multiplication. The difference is illustrated in the equations below, were you are to implement the equations in the column to the right.

GRU:

\begin{align}
&Update \: gate: \qquad &\Gamma^u=\sigma(x_tW^{u} + h_{t-1}U^{u} + b^u) \qquad \rightarrow \qquad &\Gamma^u=\sigma([x_t, h_{t-1}]W^{u} + b^u) \\
&Reset \: gate: \qquad &\Gamma^r=\sigma(x_tW^{r} + h_{t-1}U^{r} + b^r) \qquad \rightarrow \qquad &\Gamma^r=\sigma([x_t, h_{t-1}]W^{r} + b^r) \\
&Candidate \:cell: &\tilde{h_t} = tanh([x_tW + (\Gamma^r \circ h_{t-1})U + b) \qquad \rightarrow \qquad &\tilde{h_t} = tanh([x_t, (\Gamma^r \circ h_{t-1})] W + b) \\
&Final\: cell:    &h_t = \Gamma^u \circ h_{t-1} + (1-\Gamma^u) \circ \tilde{h_t} \qquad \rightarrow \qquad &h_t = \Gamma^u \circ h_{t-1} + (1-\Gamma^u) \circ \tilde{h_t}
\end{align}




In [ ]:
#Import modules
import tensorflow as tf
from sourceFiles import RNN_network
from unit_tests import unit_tests

#Resetting the tensorflow graph
tf.reset_default_graph()

#Defining dummy variables
hidden_state_sizes = 512
inputSize = 648
ind = 0

# You should implement this function
cell   = RNN_network.GRUcell(hidden_state_sizes, inputSize, ind)

#Check implementation
unit_tests.checkGRUcell(cell)

---

**Task 1h** 

You are now to implement the function "buildRNN". Use the "is_training" flag to choose between training and testing mode. Tensorflow can be slow with dynamic routing, so you are to build different static graphs for training and testing. See figure 2a and 2b. The two cells below will test your implementation in training and testing mode.



In [ ]:
#Import modules
import tensorflow as tf
from sourceFiles import RNN_network
from unit_tests import unit_tests

#Resetting the tensorflow graph
tf.reset_default_graph()

#Config
cellType            = 'GRU'   # of RNN or GRU
is_training         = True    #True/False

networkConfig, inputs, initial_states, wordEmbedding, W_hy, b_hy, batch_size = unit_tests.getConfig(cellType)

# You should implement this function
logits_series, predictions_series, current_state, predicted_tokens = RNN_network.buildRNN(networkConfig, inputs, initial_states, wordEmbedding, W_hy, b_hy, is_training)


tests = ['check_current_state', 'check_logits_series', 'predictions_series']
unit_tests.checkRNNbuild(networkConfig, tests, is_training, batch_size)

In [ ]:
#Import modules
import tensorflow as tf
from sourceFiles import RNN_network
from unit_tests import unit_tests

#Resetting the tensorflow graph
tf.reset_default_graph()

#Config
cellType            = 'GRU'   # of RNN or GRU
is_training         = False    #True/False

networkConfig, inputs, initial_states, wordEmbedding, W_hy, b_hy, batch_size = unit_tests.getConfig(cellType)

# You should implement this function
logits_series, predictions_series, current_state, predicted_tokens = RNN_network.buildRNN(networkConfig, inputs, initial_states, wordEmbedding, W_hy, b_hy, is_training)


tests = ['predicted_tokens']
unit_tests.checkRNNbuild(networkConfig, tests, is_training, batch_size)

<a id='Task2'></a>

----


# Task 2: Train the image captioning network #


Congratulations, you are done with all implemention. You shall now train various network architectures. The weights will be stored regularly, but updated when the validation loss has decrease only. When you are done training you can go to task 3 to start generating captions. 

Depending on your compute power, the training can take a long time. Start with small simple network architectures.


Tips: Do not use GradientDescent optimizer


In [ ]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
import matplotlib.pyplot as plt
%matplotlib notebook
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'


In [ ]:
import train_coco
import tensorflow as tf
from ipywidgets import IntProgress

tf.reset_default_graph()

#training config
trainingConfig   = {}
trainingConfig['trainBatchSize'] = 256
trainingConfig['valBatchSize']   = 256
trainingConfig['learning_rate']  = 0.001
trainingConfig['numberOfEpochs'] = 10
trainingConfig['getLossOnValSetEvery#epoch'] = 0.2
trainingConfig['saveModelEvery#epoch']       = 1
trainingConfig['optimizerType']              = 'RMSProp'   #Adam, RMSProp, GradientDescent
trainingConfig['inNotebook'] = True


#Network config
networkConfig = {}
networkConfig['VggFc7Size']      = 4096 #Fixed, do not change
networkConfig['embedding_size']  = 128
networkConfig['vocabulary_size'] = 4000
networkConfig['truncated_backprop_length'] = 20
networkConfig['hidden_state_sizes'] = 128
networkConfig['num_layers']         = 1
networkConfig['is_training']        = True  #
networkConfig['cellType']           = 'RNN' #RNN or GRU

#Create an instance of the "trainClass"
myTrainClass = train_coco.trainClass(trainingConfig, networkConfig)

#Path if you work on personal computer
#path = 'data/coco/'

#Path if you work on UIO IFI computer
path = /projects/anne/p22/inf5860data/

myTrainClass.myCocoDataClass.setDatapath(path)

myTrainClass.myCocoDataClass.getfilePaths()

myTrainClass.buildGraph()

myTrainClass.loss()

myTrainClass.optimizer()

myTrainClass.tensorboard()

myTrainClass.configureSession()

myTrainClass.saveWeights()

myTrainClass.trainingLoop()

<a id='Task3'></a>

---

# Task 3: Generate image captions on the validation set#

In [ ]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'

In [ ]:
import train_coco
import tensorflow as tf

tf.reset_default_graph()

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2


#training config
trainingConfig   = {}
trainingConfig['trainBatchSize'] = 256
trainingConfig['valBatchSize']   = 1
trainingConfig['learning_rate']  = 0.001
trainingConfig['numberOfEpochs'] = 10
trainingConfig['getLossOnValSetEvery#epoch'] = 0.2
trainingConfig['saveModelEvery#epoch']       = 1
trainingConfig['optimizerType']              = 'RMSProp'   #Adam, RMSProp, GradientDescent
trainingConfig['inNotebook'] = True


#Network config
networkConfig = {}
networkConfig['VggFc7Size']      = 4096 #Fixed, do not change
networkConfig['embedding_size']  = 128
networkConfig['vocabulary_size'] = 4000
networkConfig['truncated_backprop_length'] = 20
networkConfig['hidden_state_sizes'] = 128
networkConfig['num_layers']         = 1
networkConfig['is_training']        = False  #
networkConfig['cellType']           = 'RNN' #RNN or GRU

myTrainClass = train_coco.trainClass(trainingConfig, networkConfig)

#Path if you work on personal computer
#path = 'data/coco/'

#Path if you work on UIO IFI computer
path = /projects/anne/p22/inf5860data/

myTrainClass.myCocoDataClass.setDatapath(path)

myTrainClass.myCocoDataClass.getfilePaths()

myTrainClass.buildGraph()

myTrainClass.configureSession()

myTrainClass.restoreWeights()



You can run the cell below to generate captions.

In [ ]:
myTrainClass.plotImagesAndCaptions()
